# Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns

### Importando Bibliotecas e Base de Dados

In [3]:
import pandas as pd
import pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split

In [ ]:
meses = {'jan':1, 'fev';2, 'mar':3, 'abr':4, 'mai':5, 'jun':6, 'jul':7, 'ago':8, 'set':9, 'out':10, 'nov':11, 'dez':12}

caminho_dados = pathlib.Path('dataset')

dados_airbnb = pd.DataFrame()

for arquivo in caminho_dados.iterdir():
    nome_mes = arquivo.name[:3]
    mes = meses[nome_mes]
    
    ano = arquivo.name[-8:]
    ano = int(ano.replace('.csv', ''))
    
    df = pd.read_csv(caminho_dados / arquivo.name)
    df['ano'] = ano
    df['mes'] = mes
    dados_airbnb = dados_airbnb.append(df)

### Consolidar Base de Dados
* Como há muitas colunas, o modelo pode acavar ficando lento em demasia.
* Uma análise rápida constata que, várias colunas podem não ser necessárias para o modelo de previsão. Por isso será excluída algumas colunas.
* Tipos de colunas consideradas para exclusão:
    1. IDs, Links e informações não relevantes para o modelo
    2. Colunas repetidas ou extremamente parecidas com outra, que oferecem a mesma informação para o modelo. Ex: Data x Ano/Mês
    3. Coluna preenchidas com texto livre. Afinal, não será rodada nenhuma análise de palavras ou algo similar.
    4. Colunas que todos, ou quase todos, os valores sejam iguais.

### Após a análise qualitativa, considerando os critérios explicados acima, permanecem as seguintes colunas:

In [ ]:
colunas = ['host_response_time','host_response_rate','host_is_superhost','host_listings_count','latitude','longitude',\
           'property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price',\
           'security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights','maximum_nights',\
           'number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness',\
           'review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value',\
           'instant_bookable','is_business_travel_ready','cancellation_policy','ano','mes']

dados_airbnb = dados_airbnb.loc[:, colunas]

### Tratando valores faltantes
* Colunas com mais de 300.000 valres NaN foram excluídas da análise
* Para as demais colunas, como há muitos dados (mais de 900.000 linhas), foram excluídas as linhas que contém valores NaN.

In [ ]:
for coluna in dados_airbnb:
    if dados_airbnb[coluna].isnull().sum() > 300000:
        dados_airbnb = dados_airbnb.drop(coluna, axis = 1)

dados_airbnb = dados_airbnb.dropna()

print(dados_airbnb.shape)
print(dados_airbnb.isnull().sum())

### Verificar o tipo de dados em cada coluna

In [ ]:
print(dados_airbnb.dtypes)
print('-' * 60)
print(base_airbnb.iloc[0])

### Alterar as colunas preco e extra_people de object para float

In [ ]:
dados_airbnb['price'] = dados_airbnb['price'].str.replace('$', '')
dados_airbnb['price'] = dados_airbnb['price'].str.replace(',', '')
dados_airbnb['price'] = dados_airbnb['price'].astype(np.float32, copy = False)

dados_airbnb['extra_people'] = dados_airbnb['extra_people'].str.replace('$', '')
dados_airbnb['extra_people'] = dados_airbnb['extra_people'].str.replace(',', '')
dados_airbnb['extra_people'] = dados_airbnb['extra_people'].astype(np.float32, copy = False)

## Análise Exploratória e Tratar Outliers

* Observar cada feature para:
 1. Ver a correlação entre as features e decidir em manter as atuais features
 2. Excluir outliers (usando como regra, valores abaixo de Q1 - 1.5x Amplitude e valores acima de Q3 + 1.5x Amplitude). Sendo que amplitude = Q3 - Q1
 3. Confirmar se todas as features que fazem realmente sentido  ao modelo ou se deve excluir.

In [ ]:
plt.figure(figsize = (15, 10))
sns.heatmap(dados_airbnb.corr(), annot = True, cmap = 'Blues')

#### Definindo funções para análise de Outliers

In [ ]:
def limites(coluna):
    q1 = coluna.quantile(0.25)
    q3 = coluna.quantile(0.75)
    amplitude = q3 - q1
    return q1 - 1.5 * amplitude, q3 + 1.5 * amplitude

def excluir_outliers(df, nome_coluna):
    qtde_linhas = df.shape[0]
    limite_inferior, limite_superior = limites(df[nome_coluna])
    df = df.loc[(df[nome_coluna] >= limite_inferior) & (df[nome_coluna] <= limite_superior),:]
    linhas_removidas = qtde_linhas - df.shape[0]
    return df, linhas_removidas

In [ ]:
def diagrama_caixa(coluna):
    fig, (ax1, ax2) = plt.subplot(1, 2)
    fig.set_size_inches(15, 5)
    sns.boxplot(x = coluna, ax = ax1)
    ax2,set_xlim(limites(coluna))
    sns.boxplot(x = coluna, ax = ax2)
    
def histograma(coluna):
    plt.figure(figsize = (15, 5))
    sns.fistplot(coluna, hist = True)
    
def grafico_barra(coluna):
    plt.figure(figsize = (15, 5))
    ax = sns.barplot(x = coluna.value_counts().index, y = coluna.value_counts())
    ax.set_xlim(limites(coluna))

#### Colunas

##### price

In [ ]:
diagrama_caixa(dados_airbnb['price'])
histograma(dados_airbnd['price'])

Por estar sendo elaborado um modelo para imóveis comuns, acredito que os valores acima do limite superior serão apenas de apartamentos de altíssimo luxo. O que não é o objetivo principal. Por isso, optarei por excluir esses outliers.

In [ ]:
dados_airbnb, linhas_removidas = excluir_outliers(dados_airbnb, 'price')
print('{} linhas removidas'.format(linhas_removidas))

In [ ]:
histograma(dados_airbnb['price'])
print(dados_airbnb.shape)

#### extra_people

In [ ]:
diagrama_caixa(dados_airbnb['extra_people'])
histograma(dados_airbnb['extra_people'])

In [ ]:
dados_airbnb, linhas_removidas = excluir_outliers(dados_airbnb, 'extra_people')
print('{} linhas removidas'.format(linhas_removidas))

#### host_listings_count

In [ ]:
diagrama_caixa(dados_airbnb['host_listings_count'])
grafico_barra(dados_airbnb['host_listings_count'])

Excluirei esses outliers porquê, para o objetivo deste modelo, hosts com mais de seis imóveis no Airbnb, não são o público alvo do objetivo (podem ser imobiliários ou profissionais que ferenciam imóveis no Airbnb, por exemplo).

In [ ]:
dados_airbnb, linhas_removidas = excluir_outliers(dados_airbnb, 'host_listings_count')
print('{} linhas removidas'.format(linhas_removidas))

#### accommodates

In [ ]:
diagrama_caixa(dados_airbnb['accommodates'])
grafico_barra(dados_airbnb['accommodates'])

In [ ]:
dados_airbnb, linhas_removidas = excluir_outliers(dados_airbnb, 'accommodates')
print('{} linhas removidas'.format(linhas_removidas))

#### bathrooms

In [ ]:
diagrama_caixa(dados_airbnb['bathrooms'])
plt.figure(figsize = (15, 5))
sns.barplot(x = dados_airbnb['bathrooms'].value_counts().index, y = dados_airbnb['bathrooms'].value_counts())

In [ ]:
dados_airbnb, linhas_removidas = excluir_outliers(dados_airbnb, 'bathrooms')
print('{} linhas removidas'. format(linhas_removidas))

#### bedrooms

In [ ]:
diagrama_caixa(dados_airbnb['bedrooms'])
grafico_barra(dados_airbnb['bedrooms'])

In [ ]:
dados_airbnb, linhas_removidas = excluir_outliers(dados_airbnb, 'bedrooms')
print('{} linhas removidas'.format(linhas_removidas))

#### beds

In [ ]:
diagrama_caixa(dados_airbnb['beds'])
grafico_barra(dados_airbnb['beds'])

In [ ]:
dados_airbnb, linhas_removidas = excluir_outliers(dados_airbnb, 'beds')
print('{} linhas removidas'.format(linhas_removidas))

#### guests_included

In [ ]:
print(limites(dados_airbnb['guests_included']))

# por intermédio da função que foi criada, os dados estavam contra-intuitivos
plt.figure(figsize = (15, 5)) 
sns.barplot(x = dados_airbnb['guests_included'].value_counts().index, y = dados_airbnb['guests_included'].value_counts())

Optarei por remover essa feature da análise. Parece que os usuários do Airbnb usam muito o valor padrão do Airbnb como 1 guests_included.

In [ ]:
dados_airbnb = dados_airbnb.drop('guests_included', axis = 1)
dados_airbnb.shape

#### minimum_nights

In [ ]:
diagrama_caixa(dados_airbnb['minimum_nights'])
grafico_barra(dados_airbnb['minimum_nights'])

In [ ]:
dados_airbnb, linhas_removidas = excluir_outliers(dados_airbnb, 'minimum_nights')
print('{} linhas removidas'.format(linhas_removidas))

#### maximum_nights

In [ ]:
diagrama_caixa(dados_airbnb['maximum_nights'])
grafico_barra(dados_airbnb['maximum_nights'])

A demonstrar em como os dados foram preenchidos (sem a maior preocupação), considerei melhor a exclusão da coluna.

In [ ]:
dados_airbnb = dados_airbnb.drop('maximum_nights', axis = 1)
dados_airbnb.shape

#### number_of_reviews

In [ ]:
diagrama_caixa(dados_airbnb['number_of_reviews'])
grafico_barra(dados_airbnb['number_of_reviews'])

A métrica de reviews não foi considerada significativa para o objetivo

In [ ]:
dados_airbnb = dados_airbnb.drop('number_of_reviews', axis = 1)
dados_airbnb.shape

### Tratamento de Colunas de Valores de Texto

#### property_type

In [ ]:
print(dados_airbnb['property_type'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('property_type', data = dados_airbnb)
grafico.tick_params(axis = 'x', rotation = 90)

Agrupar para diminuir a quantidade de categorias

In [ ]:
tabela_tipos_casa = dados_airbnb['property_type'].value_counts()
colunas_agrupar = []

for tipo in tabela_tipos_casa.index:
    if tabela_tipo_casa[tipo] < 2000:
        colunas_agrupar.append(tipo)
print(colunas_agrupar)

for tipo in colunas_agrupar:
    dados_airbnb.loc[dados_airbnb['property_type'] == tipo, 'property_type'] = 'Outros'
    
print(dados_airbnb['property_type'].value_counts)
plt.figure(figsize = (15, 5))
grafico = sns.countplot('property_type', data = dados_airbnb)
grafico.tick_params(axis = 'x', rotation = 45)

#### room_type

In [ ]:
print(dados_airbnb['room_type'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('room_type', data = dados_airbnb)
grafico.tick_params(axis = 'x')

#### bed_type

In [ ]:
print(dados_airbnb['bed_type'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('bed_type', data = dados_airbnb)
grafico.tick_params(axis = 'x')

# agrupando categorias de bed_type

tabela_bed_type = dados_airbnb['bed_type'].value_counts()
colunas_agrupar = []

for tipo in tabela_bed_type.index:
    if tabela_bed_type[tipo] < 10000:
        colunas_agrupar.append(tipo)
print(colunas_agrupar)

for tipo in colunas_agrupar:
    dados_airbnb.loc[dados_airbnb['bed_type'] == tipo, 'bed_type'] = 'Outros'
    
print(dados_airbnb['bed_type'].value_counts())
plt.figure(figsize = (5, 5))
grafico = sns.countplot('bed_type', data = dados_airbnb)
grafico.tick_params(axis = 'x', rotation = 45)

#### cancellation_policy

In [ ]:
print(dados_airbnb['cancellation_policy'].value_counts())

plt.figure(figsize = (15, 5))
grafico = sns.countplot('cancellation_policy', data = dados_airbnb)
grafico.tick_params(axis = 'x')

# agrupando categorias de cancellation_policy

tabela_cancellation = dados_airbnb['cancellation_policy'].value_counts()
colunas_agrupar = []

for tipo in tabela_cancellation.index:
    if tabela_cancellation[tipo] < 10000:
        colunas_agrupar.append(tipo)
print(colunas_agrupar)

for tipo in colunas_agrupar:
    dados_airbnb.loc[dados_airbnb['cancellation_policy'] == tipo, 'cancellation_policy'] = 'strict'
    
print(dados_airbnb['cancellation_policy'].value_counts())
plt.figure(figsize = (15, 5))
grafico = sns.countplot('cancellation_policy', data = dados_airbnb)
grafico.tick_params(axis = 'x', rotation = 45)

#### amenities

Como é grande a diversidade de amenities e, as vezes, as mesmas amenities podem de estar nomeadas de forma diferente. Melhor avaliar a quantidade de amenities como o parâmetro para o modelo.

In [ ]:
print(dados_airbnb['amenities'].loc[1].split(',')) # igualifica todos
print(len(dados_airbnb['amenities'].iloc[1].split(',')))

dados_airbnb['n_amenities'] = dados_airbnb['amenities'].str.split(',').apply(len)

In [ ]:
dados_airbnb = dados_airbnb.drop('amenities', axis = 1)
dados_airbnb.shape

* amenities passou a ser uma coluna numérica. Desta maneira, pode receber a mesma tratativa de outras colunas numéricas.

In [ ]:
diagrama_caixa(dados_airbnb['n_amenities'])
grafico_barra(dados_airbnb['n_amenities'])

In [ ]:
dados_airbnb, linhas_removidas = excluir_outliers(dados_airbnb, 'n_amenities')
print('{} linhas removidas'.format(linhas_removidas))

## Visualização de Mapa das Propriedades

In [ ]:
amostra = dados_airbnb.sample(n = 50000)
centro_mapa = {'lat':amostra.latitude.mean(), 'lon':amostra.longitude.mean()}
fig = px.density_mapbox(amostra, lat = 'latitude', lon = 'longitude', z = 'price', radius = 2.5,
                       center = centro_mapa, zoom = 10,
                       mapbox_style = 'stamen-terrain') 
fig.show()

## Encoding